In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import t
from datetime import datetime, timedelta

In [ ]:

# Define the stock and market index
stock_ticker = 'AAPL'  # Example: Apple Inc.
market_index_ticker = '^GSPC'  # Example: S&P 500

# Define the event date and windows
event_date = datetime(2020, 1, 1)
estimation_window_start = event_date - timedelta(days=365)
estimation_window_end = event_date - timedelta(days=30)
event_window_start = event_date - timedelta(days=10)
event_window_end = event_date + timedelta(days=10)

# Fetch historical data
stock_data = yf.download(stock_ticker, start=estimation_window_start, end=event_window_end)
market_data = yf.download(market_index_ticker, start=estimation_window_start, end=event_window_end)

# Calculate daily returns
stock_returns = stock_data['Adj Close'].pct_change().dropna()
market_returns = market_data['Adj Close'].pct_change().dropna()

# Estimate the market model (CAPM) during the estimation window
estimation_window = (stock_returns.index >= estimation_window_start) & (stock_returns.index <= estimation_window_end)
X = sm.add_constant(market_returns[estimation_window])  # Market returns
y = stock_returns[estimation_window]  # Stock returns
model = sm.OLS(y, X).fit()

# Calculate expected and abnormal returns during the event window
event_window = (stock_returns.index >= event_window_start) & (stock_returns.index <= event_window_end)
expected_returns = model.params[0] + model.params[1] * market_returns[event_window]
abnormal_returns = stock_returns[event_window] - expected_returns

# Output the results
print(abnormal_returns)

# Plotting the abnormal returns
plt.figure(figsize=(12, 6))
abnormal_returns.plot()
plt.title(f'Abnormal Returns for {stock_ticker} Around the Event')
plt.xlabel('Date')
plt.ylabel('Abnormal Return')
plt.axvline(x=event_date, color='red', linestyle='--')  # Event date line
plt.show()

In [ ]:

def run_event_study(stock_ticker, event_date):
    # Define the market index
    market_index_ticker = '^GSPC'  # S&P 500

    # Define windows
    estimation_window_start = event_date - timedelta(days=365)
    estimation_window_end = event_date - timedelta(days=30)
    event_window_start = event_date - timedelta(days=10)
    event_window_end = event_date + timedelta(days=10)

    # Fetch historical data
    stock_data = yf.download(stock_ticker, start=estimation_window_start, end=event_window_end)
    market_data = yf.download(market_index_ticker, start=estimation_window_start, end=event_window_end)

    # Calculate daily returns
    stock_returns = stock_data['Adj Close'].pct_change().dropna()
    market_returns = market_data['Adj Close'].pct_change().dropna()

    # Estimate the market model during the estimation window
    estimation_window = (stock_returns.index >= estimation_window_start) & (stock_returns.index <= estimation_window_end)
    X = sm.add_constant(market_returns[estimation_window])
    y = stock_returns[estimation_window]
    model = sm.OLS(y, X).fit()

    # Calculate expected and abnormal returns during the event window
    event_window = (stock_returns.index >= event_window_start) & (stock_returns.index <= event_window_end)
    expected_returns = model.params[0] + model.params[1] * market_returns[event_window]
    abnormal_returns = stock_returns[event_window] - expected_returns

    # Calculate the average abnormal return
    average_abnormal_return = abnormal_returns.mean()

    # # Plotting
    # plt.figure(figsize=(12, 6))
    # abnormal_returns.plot(title=f'Abnormal Returns for {stock_ticker} Around the Event')
    # plt.axvline(x=event_date, color='red', linestyle='--')  # Event date line
    # plt.axhline(y=average_abnormal_return, color='blue', linestyle='--', label='Average Abnormal Return')  # Average line
    # plt.xlabel('Date')
    # plt.ylabel('Abnormal Return')
    # plt.legend()
    # plt.show()

    # Display the average abnormal return
    print(f"Average Abnormal Return for {stock_ticker}: {average_abnormal_return}")

    return abnormal_returns
# def run_event_study(stock_ticker, event_date):
#     # [Previous code parts remain unchanged]

#     # Return the abnormal returns
#     return abnormal_returns

# Example usage
stock_list = ['AAPL', 'MSFT', 'GOOGL']  # List of stocks to analyze
event_date = datetime(2018, 6, 28)  # Example event date

# Dictionary to hold abnormal returns for each stock
all_abnormal_returns = {}

for stock in stock_list:
    print(f"Event study for {stock}:")
    all_abnormal_returns[stock] = run_event_study(stock, event_date)

# Calculate average abnormal returns across stocks for each day
average_abnormal_returns = pd.concat(all_abnormal_returns, axis=1).mean(axis=1)

# Calculate the standard error of the average abnormal returns
std_error = pd.concat(all_abnormal_returns, axis=1).std(axis=1) / np.sqrt(len(stock_list))

# Determine the t-statistic for a 95% confidence interval
t_stat = t.ppf(1-0.025, df=len(stock_list)-1)

# Calculate the confidence intervals
ci_lower = average_abnormal_returns - (t_stat * std_error)
ci_upper = average_abnormal_returns + (t_stat * std_error)

# Plotting
plt.figure(figsize=(12, 6))
average_abnormal_returns.plot(title='Average Abnormal Returns with 95% Confidence Interval Across AAPL, MSFT, GOOGL Around the Event')
plt.fill_between(average_abnormal_returns.index, ci_lower, ci_upper, color='grey', alpha=0.3, label='95% Confidence Interval')
plt.axvline(x=event_date, color='red', linestyle='--')  # Event date line
plt.xlabel('Date')
plt.ylabel('Average Abnormal Return')
plt.legend()
plt.show()
